In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
import pandas as pd
import re 
import torch
import transformers 
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from transformers import AutoModel, AutoTokenizer 

from transformers import pipeline


In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_data_path = "/content/drive/MyDrive/Colab Notebooks/webis-clickbait-22/train.jsonl"
df = pd.read_json(train_data_path, lines=True)

train_data = df[['postText','targetParagraphs', 'spoiler', 'postPlatform', 'tags']].astype(str)

#train_data['postText_str'] = [''.join(map(str, l)) for l in train_data['postText']]
#train_data['targetParagraphs_str'] = [''.join(map(str, l)) for l in train_data['targetParagraphs']]
#train_data['spoiler_str'] = [''.join(map(str, l)) for l in train_data['spoiler']]

#train_data[['postText_str', 'targetParagraphs_str', 'spoiler_str']] = train_data[['postText_str', 'targetParagraphs_str', 'spoiler_str']].replace(r'\W', ' ', regex = True)

#train_data = train_data.applymap(str)

#train_data['postText_str'] = train_data['postText_str'].astype(str)
#train_data['targetParagraphs_str'] = train_data['targetParagraphs_str'].astype(str)
#train_data['spoiler_str'] = train_data['spoiler_str'].astype(str)

In [5]:
#whats the optimal length / stride?

#model = "deepset/roberta-base-squad2"
# tokenizer = AutoTokenizer.from_pretrained(model)
max_length = 500 #maximum feature length (question or context)
doc_stride = 128# authorized overlap between two parts of the context when splitting is needed 

In [11]:
def spoiler_creation(df, start, end):
  question_answer_pipeline = pipeline("question-answering", "deepset/roberta-base-squad2", tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2"), max_length=500
                                    , truncation=True, return_overflowing_tokens=True, stride=doc_stride, top_k=5) # truncation = "only_second"
  df['model_prediction'] = ""
  df['correct'] = "False"

  for i in range(start, end):
    question = df.loc[i]['postText']
    context = df.loc[i]['targetParagraphs']
    answer = df.loc[i]['spoiler']
    tags = df.loc[i]['tags']
    # if the spoiler is a phrase, the first model prediction i used
    candidates = question_answer_pipeline(question = question, context = context)
    if(tags == "['phrase']"):
      results = "['"+candidates[0]['answer']+"']"
      df.loc[i]['model_prediction'] = results
    # if the spoiler is a multi, the current top spoiler will be removed from the context and the pipeline will be rerun. 
    elif(tags == "['multi']"):
      current_context = context
      results = []
      for j in range(0,5):
        current_context = current_context
        candidates = question_answer_pipeline(question = question, context = current_context)
        current_result = candidates[0]['answer']
        results.append(current_result)
        current_context = re.sub(current_result, '', current_context)
      df.loc[i]['model_prediction'] = results
    # if the spoiler is a passage, the whole sentence containing the spoiler will be returned
    else:
      for sentence in context.split('.'):
        if candidates[0]['answer'] in sentence:
          sentence = "['"+sentence+"']"
          sentence = re.sub("(',)", "", sentence);
          sentence = re.sub('(",)', "", sentence);
          sentence = re.sub("(' )", "", sentence);
          df.loc[i]['model_prediction'] = sentence
        #else:
        #  result = "['"+result['answer']+"']"
        #  df.loc[i]['model_prediction'] = "['"+result['answer']+"']"
   
   
    if(df.loc[i]['model_prediction'] == df.loc[i]['spoiler']):
      df.loc[i]['correct'] = 'True'
    else:
      df.loc[i]['correct'] = 'False'
  
  return df

In [12]:
df_qa = train_data[['postText', 'targetParagraphs', 'spoiler', 'postPlatform', 'tags']]
df = spoiler_creation(df_qa, 30, 50)

In [13]:
df[30:50]

,postText,targetParagraphs,spoiler,postPlatform,tags,model_prediction,correct
30,['The cheapest place for a last-minute half-te...,['Cyprus is the cheapest option for a last-min...,['Cyprus'],Twitter,['phrase'],['Cyprus'],True
31,['Google paid HOW MUCH in overseas taxes!?'],"['LONDON, Sept 30 - Google, which has been gri...",['UK tax bill of 35 million pounds ($55 millio...,Twitter,['phrase'],['11.6 million pounds'],False
32,['He Brings His Girlfriend To The Hospital To ...,['Among the greatest feelings in the world are...,['was not pregnant'],Facebook,['phrase'],['convinces herself that she’s actually pregna...,False
33,['Easily Stressed People Are Mentally Healthie...,['Most successful businesses operate by follow...,['own personal values'],Facebook,['phrase'],['connecting our personal values to our experi...,False
34,['Marijuana Users Are Taking Pictures of This ...,['The Drug Enforcement Administration says mar...,"['one patent, number 6630507, disqualifies can...",reddit,['passage'],"[""To highlight the seeming contradiction, they...",False
35,"[""The summer's 1st real rom-com's about aborti...",['Romantic comedies have become so rare that w...,"['""Obvious Child""']",Twitter,['phrase'],"[''""I don\'t even think about those things at ...",False
36,['THIS IS NOT A DRILL: you could work for Kate...,['Think you have what it takes to be in the in...,['housekeeper'],Twitter,['phrase'],['housekeeper'],True
37,['Is student loan debt really stopping people ...,['Does student debt derail young people from t...,['student debt does delay major life decisions...,reddit,['passage'],[His organization has found that student debt ...,False
38,['This Is What Bill Clinton Will Actually Be C...,['If Hillary Clinton triumphs over Donald to b...,['the first gentleman'],reddit,['phrase'],['the spouse of the President of the United St...,False
39,"[""Why you shouldn't drive slowly in the left l...","[""There are basically two types of drivers: th...","[""it'll inevitably lead to buildups of traffic...",reddit,['passage'],[It's not that you're never allowed in the lef...,False
